In [18]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import the API key
from config import geoapify_key

In [19]:
# Load the restaurant ethnicities into a DataFrame
types_df = pd.read_csv("Office.csv")

# Display sample data
types_df.head()

,Office
0,company
1,estate_agent
2,insurance
3,lawyer
4,telecommunication


In [20]:
# Set up additional columns to hold information
types_df["name"] = ""
types_df["address"] = ""
types_df["distance"] = ""

# Display sample data
types_df.head()

,Office,name,address,distance
0,company,,,
1,estate_agent,,,
2,insurance,,,
3,lawyer,,,
4,telecommunication,,,


In [21]:
# Set the geographical coordinates
latitude = 42.0723
longitude = 87.7228

# Set the parameters for the type of search
limit = 20
radius = 1000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [22]:
# Print a message to follow up the restaurant search
print("Starting office search")

# Iterate through the types_df DataFrame
for index, row in types_df.iterrows():
    
    # Get the ethnicity type from the current DataFrame's row
    office = types_df.loc[index, "Office"]
   
    # Add the current ethnicity type to the parameters
    params["categories"] = f"office.{office}"
   
    # Make an API request using the params dictionary
    office = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    office = office.json()
    print(office)
    # Grab the first restaurant from the results and store the details in the DataFrame
    try:
        types_df.loc[index, "name"] = office["features"][0]["properties"]["name"]
        types_df.loc[index, "address"] = office["features"][0]["properties"]["address_line2"]
        types_df.loc[index, "distance"] = int(office["features"][0]["properties"]["distance"])
    except (KeyError, IndexError):
        # If no restaurant is found, set the restaurant name as "No restaurant found".
        types_df.loc[index, "name"] = "No office found"
        # Set the distance column value to np.nan to allow sorting values
        types_df.loc[index, "distance"] = np.nan
        
    # Log the search results
    print(f"nearest {types_df.loc[index, 'Office']} Office: {types_df.loc[index, 'name']}")

# Display sample data
types_df

Starting office search
{'type': 'FeatureCollection', 'features': []}
nearest company Office: No office found
{'type': 'FeatureCollection', 'features': []}
nearest estate_agent Office: No office found
{'type': 'FeatureCollection', 'features': []}
nearest insurance Office: No office found
{'type': 'FeatureCollection', 'features': []}
nearest lawyer Office: No office found
{'type': 'FeatureCollection', 'features': []}
nearest telecommunication Office: No office found


,Office,name,address,distance
0,company,No office found,,NaN
1,estate_agent,No office found,,NaN
2,insurance,No office found,,NaN
3,lawyer,No office found,,NaN
4,telecommunication,No office found,,NaN


In [6]:
# Sort restuls by distance
types_df = types_df.sort_values(by=["distance"])

# Display the top 10 nearest restaurants
types_df.head(10)

,OFFICE WORDS,name,address,distance
0,company,No office found,,NaN
1,estate_agent,No office found,,NaN
2,insurance,No office found,,NaN
3,lawyer,No office found,,NaN
4,telecommunication,No office found,,NaN
5,educational_institution,No office found,,NaN
6,assosciation,No office found,,NaN
